In [5]:
from utils import convert_json_to_sentence, generate_category_prompts
import pandas as pd
from dataset import PromptDataset
from clip_linear import ClipLinearProbing
from torch.utils.data import random_split, DataLoader
import torch
import numpy as np  

In [3]:
data = pd.read_json(r"/home/guillermo/ssd/Github/BreastCATT/prompts/1.json")
convert_json_to_sentence(data)

'Patient is 76 years old, widow of unspecified race, registered on 2012-10-24. Has eating habits rich in low in fat. No relevant clinical signs or symptoms were reported. The recorded body temperature is 34.90°C. No mention is made of alcohol consumption, coffee drinking, smoking, recent physical exercise.'

In [6]:
data = pd.read_json(r"/home/guillermo/ssd/Github/BreastCATT/prompts/1.json")
generate_category_prompts(data)

{'risk_factors': 'Risk factors: Age: 76; Eating habits: low in fat; Family history of cancer: ; Radiotherapy treatment: no; Hormone replacement: no; Age at menarche: ; Age at menopause: ; Diabetes history: not specified; Presence of nodules: not specified.',
 'complementary_features': 'Complementary features: Prosthesis: no; Signals of wart on breasts: yes, both breasts; Nipple changes: ; Mastectomy: not specified.',
 'protocol_features': 'Protocol features: Smoked: ; Drank coffee: ; Consumed alcohol: ; Physical exercise: ; Use of deodorant/products: .'}

In [4]:
# dataset = PromptDataset(r"C:\Users\opera\OneDrive\Documentos\GitHub\BreastCATT\prompts", r"C:\Users\opera\OneDrive\Documentos\GitHub\BreastCATT\patient_labels.json")
dataset = PromptDataset('/home/guillermo/ssd/Github/BreastCATT/prompts', '/home/guillermo/ssd/Github/BreastCATT/patient_labels.json')


# Use DataLoader to load the data in batches
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = ClipLinearProbing(device)

In [6]:
for i, batch in enumerate(dataloader):
    prompt = batch["prompt"]
    label = batch["label"]

In [7]:
prompt

['Patient is 51 years old, single of unspecified race, registered on 2013-04-11. Has eating habits rich in low in fat. Their last menstruation was at 50 years old and menarche occurred at 12 years old. No information is provided for mammography, biopsy, plastic surgery, prosthesis, radiotherapy, use of hormone replacement. No mention is made of alcohol consumption, coffee drinking, smoking, recent physical exercise.']

In [8]:
label

tensor([0])

In [13]:
# Calculate features

all_features = []
all_labels = []

for data in dataloader:
  prompt = data["prompt"]  # get the prompt string from the batch
  # print(data['prompt'][0].shape)
  label = torch.tensor(data["label"][0]).float()
  
  with torch.no_grad():
      # model expects a list of prompt(s)
      text_features = model(prompt)
  all_features.append(text_features)
  all_labels.append(label.unsqueeze(0))

all_features = (torch.sigmoid(torch.cat(all_features).cpu()) > 0.5).float().numpy()
all_labels = torch.cat(all_labels).cpu().numpy()
print(all_features.shape, all_labels.shape)

/tmp/ipykernel_11411/49885607.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(data["label"][0]).float()


(278, 1) (278,)


In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(all_features, all_labels, test_size=0.3, random_state=42)

In [30]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=0,C=0.4, max_iter=1000, verbose=1)
classifier.fit(X_train, y_train)

predictions = classifier.predict(X_test)
accuracy = np.mean((y_test == predictions).astype(float)) * 100.
print(accuracy)

61.904761904761905
